## 1. Import all the packages/libraries first! <font color="#FF0000">(Execute Only)</font>

In [ ]:
%run scripts/tfidf_cfidf_process_functions.ipynb

## 2. Modin Pandas (parallel version of Pandas) <font color="#FF0000">(Execute Only)</font>

In [ ]:
# Import modin pandas for tackling with large dataset
import modin.pandas as mpd
from modin.config import ProgressBar
ProgressBar.enable()

# Use dask as modin engine
import modin.config as cfg
cfg.Engine.put("dask")

## ================
## If you don't know what it is doing
## Don't touch!
## ================
from dask.distributed import Client
## ================
## If you have 8 cores & Working string/dictionaries data
## n_workers = 4
## threads_per_worker= 2
## ==OR==
## If you have 8 cores & Working numeric data
## n_workers = 2
## threads_per_worker= 4
## !!! The parameters are depend on what you are going to do !!!
## ================
client = Client(processes=True, n_workers=4, threads_per_worker=4)
swifter.register_modin()  # start distributed scheduler locally
client

## 3. Data pre-processing

### 3.1 Convert xlsx format to csv format (optional) <font color="#FF0000">(Execute & Configure)</font>

In [26]:
# 3.1 Convert xlsx format to csv format
# client.restart()
from datetime import datetime
column = ""
error_list = []
now = datetime.now()
error_filename = now.strftime("%Y%m%d_%H%M%S")
def loadDataset_xlsx(btn):
  global CSV_PATH
  CSV_PATH = glob.glob(path.join(folder_pathDisplay1.value, file_dropdown1.value, '*.xlsx'))
  print(f"Successfully set {CSV_PATH[0]} and the rest as dataset source")
 
def xlsxToCSV(btn):
  clear_output(wait=True)
  print('Processing!')
  for XLSX in tqdm(CSV_PATH, desc="Progress: ", ncols=100):
    dest_filename = path.basename(XLSX).split('.')[0] + '.csv'
    destination = path.join(DEST_PATH, dest_filename)
    try:
        skipRow = 1 if checkbox_skipFirstRow.value else 0
        if checkbox_header.value:
          df = pd.read_excel(XLSX, skiprows=skipRow)
          df.to_csv(destination, encoding="utf-8-sig", index=False)
        else:
          df = pd.read_excel(XLSX, header=None, skiprows=skipRow)
          df.to_csv(destination, encoding="utf-8-sig", index=False, header=None)
    except:
        error_list.append(XLSX)
        print("Error occurred")
  if len(error_list) > 0:
    print("Error list is saved to root folder")
    with open("Section3.1_" + error_filename + ".txt", "w") as file:
      file.write("\r\n".join(error_list))
  print('Done')
 
# Configure & Reload Widgets
btn_load1 = button('Load Source!')
btn_setDest = button('Select Column!')
btn_exeFx = button('Execute!')
checkbox_header = widgets.Checkbox(
    value=False,
    description='Contain header?',
    disabled=False
)
checkbox_skipFirstRow = widgets.Checkbox(
    value=False,
    description='Skip first row?',
    disabled=False
)
 
btn_load1.on_click(loadDataset_xlsx)
btn_setDest.on_click(setDestination)
btn_exeFx.on_click(xlsxToCSV)
 

tab_load1 = VBox([
  widgets.HTML(value="<b><font style='color:red'>Select & Load your dataset Folder:</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  checkbox_header,
  checkbox_skipFirstRow,
  btn_load1
])
 
tab_setDest = VBox([
  widgets.HTML(value="Select Output Destination: <b><font style='color:red'>Make sure you have created a folder first!</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_setDest
  ])
 
tab_execute = VBox([
  widgets.HTML(value='<b>If Everything is ready, Click the button below:</b>'),
  btn_exeFx
])
 
tab.children = [tab_load1, tab_setDest, tab_execute]
for index, title in enumerate([tabTitles2[0],tabTitles2[5],tabTitles2[-1]]):
  tab.set_title(index, title)
display(tab)

Processing!


Progress: 100%|███████████████████████████████████████████████████| 831/831 [00:13<00:00, 62.69it/s]

Done


### 3.2.1 Add column name (optional) <font color="#FF0000">(Execute & Configure)</font>

In [ ]:
# 3.2.1 Add column name
columns = []
tabColumns = Tab()

def loadColumns(btn):
  global columns, tabColumns
  with open(CSV_PATH[0], 'r', encoding='utf-8-sig') as file:
    columns = [column.strip('\n') for column in file.readline().split(',')]
    children = [widgets.Text(name=x) for x in columns]
    tabColumns.children = children
    for i, column in enumerate(columns):
      tabColumns.set_title(i, column)
      # tabColumns.children[i].value = column
    print(f"Successfully loaded the columns")

def setColumnsName(btn):
  global columns
  columns = [text.value for text in tabColumns.children]
  print(f"Column name: {columns}")

def addColumnName(btn):
  print('Processing')
  for CSV in tqdm(CSV_PATH, desc="Progress: ", ncols=100):
    lines = ""
    with open(CSV, 'r', encoding='utf-8-sig') as file:
      lines = file.readlines()
    lines.insert(0, ",".join(columns) + "\n")
    # lines[0] = ",".join(columns) + "\n"
    with open(CSV, 'w', encoding='utf-8-sig') as file:
      file.writelines(lines)
  print('Done')
  
# Configure & Reload Widgets
input_Text = inputText('File name:')

btn_load1 = button('Load Source!')
btn_load2 = button('Load First File!')
btn_setColumnName = button('Set!')
btn_exeFx = button('Execute!')

btn_load1.on_click(loadDataset)
btn_load2.on_click(loadColumns)
btn_setColumnName.on_click(setColumnsName)
btn_exeFx.on_click(addColumnName)

tab_load1 = VBox([
  widgets.HTML(value="<b><font style='color:red'>Select & Load your dataset Folder:</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_load1
])

tab_setColumnName = VBox([
  widgets.HTML(value="<b>Set the name of the columns!!</b>"),
  widgets.HTML(value="<b><font style='color:red'>If you want to use our tool</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>Make sure to set the text data column name to 'content'</font></b>"),
  btn_load2,
  tabColumns,
  btn_setColumnName
])

tab_execute = VBox([
  widgets.HTML(value='<b>If Everything is ready, Click the button below:</b>'),
  btn_exeFx
])

tab.children = [tab_load1, tab_setColumnName, tab_execute]
for index, title in enumerate([tabTitles2[0],tabTitles2[1], tabTitles2[-1]]):
  tab.set_title(index, title)

display(tab)

### 3.2.2 Rename columns (optional) <font color="#FF0000">(Execute & Configure)</font>

In [ ]:
# 3.2.2 Rename columns
columns = []
tabColumns = Tab()

def loadColumns(btn):
  global columns, tabColumns
  with open(CSV_PATH[0], 'r', encoding='utf-8-sig') as file:
    columns = [column.strip('\n') for column in file.readline().split(',')]
    children = [widgets.Text(name=x) for x in columns]
    tabColumns.children = children
    for i, column in enumerate(columns):
      tabColumns.set_title(i, column)
      tabColumns.children[i].value = column
    print(f"Successfully loaded the columns")

def setColumnsName(btn):
  global columns
  columns = [text.value for text in tabColumns.children]
  print(f"Column name: {columns}")

def replaceColumnName(btn):
  clear_output(wait=True)
  print('Processing')
  for CSV in tqdm(CSV_PATH, desc="Progress: ", ncols=100):
    lines = ""
    with open(CSV, 'r', encoding='utf-8-sig') as file:
      lines = file.readlines()
    lines[0] = ",".join(columns) + "\n"
    with open(CSV, 'w', encoding='utf-8-sig') as file:
      file.writelines(lines)
  print('Done')
  
# Configure & Reload Widgets
input_Text = inputText('File name:')

btn_load1 = button('Load Source!')
btn_load2 = button('Load First File!')
btn_setColumnName = button('Set!')
btn_exeFx = button('Execute!')

btn_load1.on_click(loadDataset)
btn_load2.on_click(loadColumns)
btn_setColumnName.on_click(setColumnsName)
btn_exeFx.on_click(replaceColumnName)

tab_load1 = VBox([
  widgets.HTML(value="<b><font style='color:red'>Select & Load your dataset Folder:</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_load1
])

tab_reNameColumns = VBox([
  widgets.HTML(value="<b>Set the name of the columns!!</b>"),
  widgets.HTML(value="<b><font style='color:red'>If you want to use our tool</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>Make sure to set the text data column name to 'content'</font></b>"),
  btn_load2,
  tabColumns,
  btn_setColumnName
])

tab_execute = VBox([
  widgets.HTML(value='<b>If Everything is ready, Click the button below:</b>'),
  btn_exeFx
])

tab.children = [tab_load1, tab_reNameColumns, tab_execute]
for index, title in enumerate([tabTitles2[0],tabTitles2[1], tabTitles2[-1]]):
  tab.set_title(index, title)

display(tab)

### 3.2.3 Drop columns (optional) <font color="#FF0000">(Execute & Configure)</font>

In [ ]:
# 3.2.3 Drop columns
client.restart() #Restart Dask
RANGE = 1
columnsToDrop = []

def loadColumns_1(btn):
  with open(CSV_PATH[0], 'r', encoding='utf-8-sig') as file:
    columns = [column.strip('\n') for column in file.readline().split(',')]
    columnsDropdown_1.options = columns
    print(f"Successfully loaded the columns")

def addColumnsToDrop(btn):
  global columnsToDrop
  columnsToDrop.append(columnsDropdown_1.value)
  print(f"Columns to be Ignored: {columnsToDrop}")

def resetColumns(btn):
  global columnsToDrop
  columnsToDrop.clear()
  print(f"Columns to be Ignored: {columnsToDrop}")

def dropColumns(btn):
  clear_output(wait=True)
  for CSV in tqdm(list(CSV_PATH)[RANGE-1:], desc = f"Progress", ncols=100):
  # for CSV in track(list(CSV_PATH)[RANGE-1:], description=f"Conversion Progress"):
    # Initislization
    dest_filename = path.basename(CSV).split('.')[0] + '.csv'
    destination = path.join(DEST_PATH, dest_filename)
    df = pd.read_csv(CSV, encoding='utf-8-sig')
    df = df.drop(columnsToDrop, axis=1)
    df.to_csv(destination, encoding='utf-8-sig', index=False)

# =======================================================
# =======================================================
# =======================================================

# Configure & Reload Widgets
# columnsDropdown = Dropdown([], 'Select Column: ')

btn_load1 = button('Load Source!')
btn_loadColumns_1 = button('Load Columns!')
btn_addColumn_output = button('Add to drop list!')
btn_resetColumn_output = button('Reset!')
btn_setDest = button('Set Path')
btn_setRange = button('Set Range!')
btn_exeFx = button('Execute!')

btn_load1.on_click(loadDataset)
btn_loadColumns_1.on_click(loadColumns_1)
btn_addColumn_output.on_click(addColumnsToDrop)
btn_resetColumn_output.on_click(resetColumns)
btn_setDest.on_click(setDestination)
btn_setRange.on_click(setRange)
btn_exeFx.on_click(dropColumns)

input_boundedInt = boundedInt()

tab_load1 = VBox([
  widgets.HTML(value="<b><font style='color:red'>Select & Load your dataset Folder:</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_load1
])

tab_setDropColumns = VBox([
  widgets.HTML(value="<b><font style='color:red'>This section will set the output format of your work</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>Drop unwanted columns: </font></b>"),
  btn_loadColumns_1,
  columnsDropdown_1,
  HBox([
  btn_addColumn_output,
  btn_resetColumn_output
  ])
])

tab_setDest = VBox([
  widgets.HTML(value="Select Output Destination: <b><font style='color:red'>Make sure you have created a folder first!</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_setDest
  ])

tab_setRange = VBox([
  widgets.HTML(value="Set the Starting Position for processing: <b>Default processing the <font style='color:red'>FIRST</font> file</b>"),
  input_boundedInt,
  btn_setRange
])

tab_execute = VBox([
  widgets.HTML(value='<b>If Everything is ready, Click the button below:</b>'),
  btn_exeFx
])

#Tab
tab.children = [tab_load1, tab_setDropColumns, tab_setDest, tab_setRange, tab_execute]
for index, title in enumerate([tabTitles2[0], "Drop Columns"] \
  + tabTitles[3:]):
  
  tab.set_title(index, title)

display(tab)

### 3.3 Split dataset in CSV format according to column (optional) <font color="#FF0000">(Execute & Configure)</font>

In [ ]:
# 3.3 Split dataset according to column
# client.restart()
column = ""
fileName = ""

def loadColumns_1(btn):
  with open(CSV_PATH[0], 'r', encoding='utf-8-sig') as file:
    columns = [column.strip('\n') for column in file.readline().split(',')]
    columnsDropdown.options = columns
    print(f"Successfully loaded the columns")

def selectColumn(btn):
  global column
  column = columnsDropdown.value
  print(f"Split dataset according to {column}")

def mergeToSingleCSV():
  print('Processing')
  global fileName
  fileName = path.join(folder_pathDisplay1.value, "temp_merged.csv")
  with open(fileName, 'w', encoding='utf-8-sig') as file:
    for p_index, CSV in tqdm(list(enumerate(CSV_PATH)), desc="Progress: ", ncols=100):
      with open(CSV, encoding="utf-8-sig") as f:
          for idx, line in enumerate(f):
              if p_index == 0 and idx == 0:
                file.write(line)
              elif idx>0:
                file.write(line)
  print('Done')
  
def splitCSV(btn):
  clear_output(wait=True)
  print('Processing!')
  if len(CSV_PATH) > 1:
    mergeToSingleCSV()
    df = mpd.read_csv(fileName, encoding='utf-8-sig').groupby(column)
  else:
    df = mpd.read_csv(CSV_PATH[0], encoding='utf-8-sig').groupby(column)
  # df_date = df.groupby('date')
  # DEST_PATH = path.join(DIR, 'data_full_csv_date')
  for name, dataframe in tqdm(np.array(df), desc = f"Conversion Progress", ncols=100):
    dest_filename = str(name) + '.csv'
    destination = path.join(DEST_PATH, dest_filename)
    try:
        dataframe.to_csv(destination, encoding="utf-8-sig", index=False)
    except:
        print("Error")
  print('Done')

# Configure & Reload Widgets
columnsDropdown = Dropdown([], 'Select Column: ')

btn_load1 = button('Load Source!')
btn_loadColumns_1= button('Load Columns!')
btn_selectColumn = button('Select Column!')
btn_setDest = button('Select Column!')
btn_exeFx = button('Execute!')

btn_load1.on_click(loadDataset)
btn_loadColumns_1.on_click(loadColumns_1)
btn_selectColumn.on_click(selectColumn)
btn_setDest.on_click(setDestination)
btn_exeFx.on_click(splitCSV)

tab_load1 = VBox([
  widgets.HTML(value="<b><font style='color:red'>This section will process One/Multiple CSV files, put the file(s) into a folder</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>**Multiple files: This also create a temp_merged.csv file to the root folder, you may either make good use of it or delete it</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>Select & Load your dataset Folder:</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_load1
])

tab_selectSplitColumn = VBox([
  widgets.HTML(value="<b><font style='color:red'>Split dataset according to which column? :</font></b>"),
  btn_loadColumns_1,
  columnsDropdown,
  btn_selectColumn
])

tab_setDest = VBox([
  widgets.HTML(value="Select Output Destination: <b><font style='color:red'>Make sure you have created a folder first!</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_setDest
  ])

tab_execute = VBox([
  widgets.HTML(value='<b>If Everything is ready, Click the button below:</b>'),
  btn_exeFx
])

tab.children = [tab_load1, tab_selectSplitColumn, tab_setDest, tab_execute]
for index, title in enumerate([tabTitles2[0],tabTitles2[2],tabTitles2[5],tabTitles2[-1]]):
  tab.set_title(index, title)
display(tab)

### 3.4 Data cleaning (optional) <font color="#FF0000">(Execute & Configure)</font>

In [ ]:
# 3.4 Data cleaning
#Data Cleaning function (Single)
#==========ReEx pattern==========
# URL_PATTERN = r'(?<=[^.])((?:(?:https?|ftp|file)://|(?<![a-zA-Z\-\.])www\.)' \
#               r'[\-A-Za-z0-9\+&@\(\)#/%\?=\~_|!:\,\.\;]+[\-A-Za-z0-9\+&@#/%=\~_\|])' \
#               r'(?=[<\u4E00-\u9FA5￥，。；！？、“”‘’>（）—《》…● \t\n])'
URL_PATTERN = r'((?:(?:https?|ftp|file)://|(?<![a-zA-Z\-\.])www\.)' \
              r'[\-A-Za-z0-9\+&@\(\)#/%\?=\~_|!:\,\.\;]+[\-A-Za-z0-9\+&@#/%=\~_\|])' \
              r'(?=[<\u4E00-\u9FA5￥，。；！？、“”‘’>（）—《》…● \t\n])'
# HTML Tag Clearing     
re_HTML = re.compile('<.*?>')
# Custom Characters
re_Characters = re.compile('[=&,.%;：:()（）【】［］『』「」。、，!！?？"\[\]\-\s+]+')
re_Characters2 = re.compile('[❌😭✂️👐🏻+]+')
re_Characters_All = re.compile('[\W ]+')
# Chinese & English Space Clearing
re_ChiEng_Space = re.compile('([\u4e00-\u9fff])\s+([\u4e00-\u9fff])')
re_ChiEng_Space2 = re.compile('([a-zA-Z])\s+([\u4e00-\u9fff])')
re_ChiEng_Space3 = re.compile('([\u4e00-\u9fff])\s+([a-zA-Z])')
# Numbers & Chinses Clearing
re_Number_Space = re.compile('([\u4e00-\u9fff])\s+([0-9])')
re_Number_Space2 = re.compile('([0-9])\s+([\u4e00-\u9fff])')
re_Number_Space3 = re.compile('([0-9])\s+([0-9])')
# URL Clearing
re_URL_PATTERN = re.compile(URL_PATTERN)
re_URL_PATTERN2 = re.compile(r'[a-zA-Z]+://[\S]+\.[net|com|org|info|edu|gov|uk|de|ca|jp|fr|au|us|ru|ch|it|nel|se|no|es|mil|be|hk][\S]*\s?[\u4e00-\u9fff]?')
re_EmptyCell = r'^\s*$'
#FULL_WIDTH to HALF_WIDTH
WIDTH_MAP = {'　': ' ', '！': '!', '＂': '"', '＃': '#', '＄': '$', '％': '%', '＆': '&', 
       '＇': "'", '（': '(', '）': ')', '＊': '*', '＋': '+', '，': ',', '－': '-', 
       '．': '.', '／': '/', 
       '０': '0', '１': '1', '２': '2', '３': '3', '４': '4', '５': '5', '６': '6', 
       '７': '7', '８': '8', '９': '9', 
       '：': ':', '；': ';', '＜': '<', '＝': '=', '＞': '>', '？': '?', '＠': '@',
       'Ａ': 'A', 'Ｂ': 'B', 'Ｃ': 'C', 'Ｄ': 'D', 'Ｅ': 'E', 'Ｆ': 'F', 'Ｇ': 'G',
       'Ｈ': 'H', 'Ｉ': 'I', 'Ｊ': 'J', 'Ｋ': 'K', 'Ｌ': 'L', 'Ｍ': 'M', 'Ｎ': 'N',
       'Ｏ': 'O', 'Ｐ': 'P', 'Ｑ': 'Q', 'Ｒ': 'R', 'Ｓ': 'S', 'Ｔ': 'T', 'Ｕ': 'U',
       'Ｖ': 'V', 'Ｗ': 'W', 'Ｘ': 'X', 'Ｙ': 'Y', 'Ｚ': 'Z', 
       '［': '[', '＼': '\\', 
       '］': ']', '＾': '^', '＿': '_', '｀': '`',
       'ａ': 'a', 'ｂ': 'b', 'ｃ': 'c', 'ｄ': 'd', 'ｅ': 'e', 'ｆ': 'f', 'ｇ': 'g',
       'ｈ': 'h', 'ｉ': 'i', 'ｊ': 'j', 'ｋ': 'k', 'ｌ': 'l', 'ｍ': 'm', 'ｎ': 'n',
       'ｏ': 'o', 'ｐ': 'p', 'ｑ': 'q', 'ｒ': 'r', 'ｓ': 's', 'ｔ': 't', 'ｕ': 'u',
       'ｖ': 'v', 'ｗ': 'w', 'ｘ': 'x', 'ｙ': 'y', 'ｚ': 'z', 
       '｛': '{', '｜': '|', '｝': '}'}

re_FullWidth = re.compile(r'[\u3000！＂＃＄％＆＇（）＊＋，－．／０１２３４５６７８９'\
            r'：；＜＝＞？＠ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ［＼］＾＿｀'\
            r'ａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚ｛｜｝]+')
#================================
def full_to_half(words):
    temp = ''
    if len(words) != 0:
        for word in words:
            temp += WIDTH_MAP[word]
    return temp
  
def dataClearingPipe(row):
    row = re.sub(URL_PATTERN, '', str(row)) # Clear URLs
    row = re_URL_PATTERN2.sub('', str(row)) # Clear URLs
    row = re.sub(re_HTML, '', str(row)) # Clear HTML tag
    row = re.sub(re_Characters_All, ' ', str(row)) # Clear custom characters including spaces #1
    row = re_ChiEng_Space.sub(r'\1\2', str(row)) # Clear Spaces between Chinese Words #1
    row = re_ChiEng_Space2.sub(r'\1\2', str(row)) # Clear Spaces between Eng & Chinese Words
    row = re_ChiEng_Space3.sub(r'\1\2', str(row)) # Clear Spaces between Chinese & Eng Words
    row = re_ChiEng_Space.sub(r'\1\2', str(row)) # Clear Spaces between Chinese Words #2
    row = re_Number_Space.sub(r'\1\2', str(row)) # Clear Spaces between Chinese & Numbers
    row = re_Number_Space2.sub(r'\1\2', str(row)) # Clear Spaces between Numbers & Chinese
    row = re_Number_Space3.sub(r'\1\2', str(row)) # Clear Spaces between Numbers & Numbers
    row = row.lower() # Covert All CAPITAL letters to SMALL letters
    # ==========FULL_WIDTH to HALF_WIDTH===============
    if re_FullWidth.findall(row):
        for word in re_FullWidth.findall(row):
            half_word = full_to_half(word)
            row = re.sub(word, half_word, row)
    # =================================================
    row = re.sub(re_Characters_All, ' ', str(row)) # Clear custom characters including spaces #2
    return row.strip() #Return content with trimmed head&tail

def dataCleaning(btn):  
  clear_output()
  for CSV in tqdm(list(CSV_PATH)[RANGE-1:], desc = f"Progress", ncols=100):
  # for CSV in track(list(CSV_PATH)[RANGE-1:], description=f"Conversion Progress"):
    # Initislization
    dest_filename = path.basename(CSV).split('.')[0] + '.csv'
    destination = path.join(DEST_PATH, dest_filename)
    df = pd.read_csv(CSV, encoding='utf-8-sig')
    df['content'] = df['content'].swifter.apply(dataClearingPipe)
    df = df[df['content'] != '']
    df = df[df['content'] != 'None']
    df.to_csv(destination, encoding="utf-8-sig", index=False)
  print("Done!")

def loadColumns(btn):
  with open(CSV_PATH[0], 'r', encoding='utf-8-sig') as file:
    columns = [column.strip('\n') for column in file.readline().split(',')]
    columnsDropdown.options = columns
    print(f"Successfully loaded the columns")

def selectColumn(btn):
  global column
  column = columnsDropdown.value
  print(f"Split dataset according to {column}")

columnsDropdown = Dropdown([], 'Select Column: ')
  
btn_load1 = button('Load Source!')
btn_loadColumns = button('Load Columns!')
btn_selectColumn = button('Select Column!')
btn_setDest = button('Select Column!')
btn_setRange = button('Set Range!')
btn_exeFx = button('Execute!')

btn_load1.on_click(loadDataset)
btn_loadColumns.on_click(loadColumns)
btn_selectColumn.on_click(selectColumn)
btn_setDest.on_click(setDestination)
btn_setRange.on_click(setRange)
btn_exeFx.on_click(dataCleaning)

input_boundedInt = boundedInt()

tab_load1 = VBox([
  widgets.HTML(value="<b><font style='color:red'>Select & Load your dataset Folder:</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_load1
])

tab_selectSplitColumn = VBox([
  widgets.HTML(value="<b><font style='color:red'>Split dataset according to which column? :</font></b>"),
  btn_loadColumns,
  columnsDropdown,
  btn_selectColumn
])

tab_setDest = VBox([
  widgets.HTML(value="Select Output Destination: <b><font style='color:red'>Make sure you have created a folder first!</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_setDest
  ])

tab_setRange = VBox([
  widgets.HTML(value="Set the Starting Position for processing: <b>Default processing the <font style='color:red'>FIRST</font> file</b>"),
  input_boundedInt,
  btn_setRange
])

tab_execute = VBox([
  widgets.HTML(value='<b>If Everything is ready, Click the button below:</b>'),
  btn_exeFx
])

tab.children = [tab_load1, tab_selectSplitColumn, tab_setDest, tab_setRange, tab_execute]
for index, title in enumerate([tabTitles2[0],tabTitles2[2],tabTitles2[5], "Set Range", tabTitles2[-1]]):
  tab.set_title(index, title)
display(tab)

## 4. Processing Data (i.e. Text-mining process)

### 4.0.1 Segment your text data (i.e "content") using Jieba! <font color="#FF0000">(Execute & Configure)</font>

In [ ]:
# 4.0.1 Segment text data using Jieba
column = ""

def loadColumns(btn):
  with open(CSV_PATH[0], 'r', encoding='utf-8-sig') as file:
    columns = [column.strip('\n') for column in file.readline().split(',')]
    columnsDropdown.options = columns
    print(f"Successfully loaded the columns")

def selectColumn(btn):
  global column
  column = columnsDropdown.value
  print(f"Split dataset according to {column}")

def segmentDatasetContent(btn):
  clear_output(wait=True)
  for CSV in tqdm(list(CSV_PATH)[RANGE-1:], desc = f"Conversion Progress", ncols=100):
    # for CSV in track(list(CSV_PATH)[RANGE-1:], description=f"Conversion Progress"):
    # Initislization
    dest_filename = path.basename(CSV).split('.')[0] + '.csv'
    destination = path.join(DEST_PATH, dest_filename)
    df = pd.read_csv(CSV, encoding='utf-8-sig')
    npContent = np.array(df[column])
    df[column] = [segmentation(x) for x in npContent]
    df.to_csv(destination, encoding="utf-8-sig", index=False)

# Configure & Reload Widgets
columnsDropdown = Dropdown([], 'Select Column: ')

btn_load1 = button('Load Source!')
btn_loadColumns = button('Load Columns!')
btn_selectColumn = button('Select Column!')
btn_setDest = button('Set Path')
btn_setRange = button('Set Range!')
btn_exeFx = button('Execute!')

btn_load1.on_click(loadDataset)
btn_loadColumns.on_click(loadColumns)
btn_selectColumn.on_click(selectColumn)
btn_setDest.on_click(setDestination)
btn_setRange.on_click(setRange)
btn_exeFx.on_click(segmentDatasetContent)


tab_load1 = VBox([
  widgets.HTML(value="<b><font style='color:red'>Select & Load your dataset Folder:</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_load1
])


tab_selectColumn = VBox([
  widgets.HTML(value="<b><font style='color:red'>Set column:</font></b>"),
  btn_loadColumns,
  columnsDropdown,
  btn_selectColumn
])

tab_setDest = VBox([
  widgets.HTML(value="Select Output Destination: <b><font style='color:red'>Make sure you have created a folder first!</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_setDest
  ])

tab_setRange = VBox([
  widgets.HTML(value="Set the Starting Position for processing: <b>Default processing the <font style='color:red'>FIRST</font> file</b>"),
  input_boundedInt,
  btn_setRange
])

tab_execute = VBox([
  widgets.HTML(value='<b>If Everything is ready, Click the button below:</b>'),
  btn_exeFx
])

#Tab
tab.children = [tab_load1, tab_selectColumn, tab_setDest, tab_setRange, tab_execute]
for index, title in enumerate([tabTitles[0]] + ["Select Column"] + tabTitles[3:]):
  tab.set_title(index, title)

display(tab)

### 4.0.2 Replace your segmented data with Concept <font color="#FF0000">(Execute & Configure)</font>

In [ ]:
# 4.0.2 Replace your segmented data with Concept
column = ""

def replaceSegmentedDataset(btn):
  clear_output(wait=True)
  for CSV in tqdm(list(CSV_PATH)[RANGE-1:], desc = f"Conversion Progress", ncols= 100):
  # Initialization
    dest_filename = path.basename(CSV).split('.')[0] + '.csv'
    destination = path.join(DEST_PATH, dest_filename)
    # df = mpd.read_csv(CSV, encoding='utf-8-sig')
    df = pd.read_csv(CSV, encoding='utf-8-sig')
    # npContent = np.array(df['content'])
    # df['content'] = [segment_df_cfidf(x) for x in npContent]
    df['content'] = df['content'].swifter.apply(segmented_replaceTerms_cfidf)
    df.to_csv(destination, encoding="utf-8-sig", index=False)

# Configure & Reload Widgets
columnsDropdown = Dropdown([], 'Select Column: ')

btn_load1 = button('Load Source!')
btn_setDest = button('Set Path')
btn_setRange = button('Set Range!')
btn_exeFx = button('Execute!')

btn_load1.on_click(loadDataset)
btn_setDest.on_click(setDestination)
btn_setRange.on_click(setRange)
btn_exeFx.on_click(replaceSegmentedDataset)


tab_load1 = VBox([
  widgets.HTML(value="<b><font style='color:red'>Make sure your have segmented your dataset in Section 4.0.1</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>Select & Load your segmented dataset Folder:</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_load1
])


tab_setDest = VBox([
  widgets.HTML(value="Select Output Destination: <b><font style='color:red'>Make sure you have created a folder first!</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_setDest
  ])

tab_setRange = VBox([
  widgets.HTML(value="Set the Starting Position for processing: <b>Default processing the <font style='color:red'>FIRST</font> file</b>"),
  input_boundedInt,
  btn_setRange
])

tab_execute = VBox([
  widgets.HTML(value='<b>If Everything is ready, Click the button below:</b>'),
  btn_exeFx
])

#Tab
tab.children = [tab_load1, tab_setDest, tab_setRange, tab_execute]
for index, title in enumerate([tabTitles[0]] + tabTitles[3:]):
  tab.set_title(index, title)

display(tab)

### 4.0.3 Load Your Segmented Dataset (done in section 4.0.1 & 4.0.2) <font color="#FF0000">(Execute & Configure)</font>

In [ ]:
# 4.0.2 load segmented dataset
foldersOnly = list(filter(lambda x: path.isdir(x), listdir(DIR)))
# Configure & Reload Widgets
btn_load1 = button('Load Source!')
btn_load2 = button('Load Source!')

btn_load1.on_click(loadDataset_clear)
btn_load2.on_click(loadDataset_clear2)

tab_load1 = VBox([
  widgets.HTML(value="<b><font style='color:red'>Select & Load your 4.0.1 dataset Folder:</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_load1
])

tab_load2 = VBox([
  widgets.HTML(value="<b><font style='color:red'>Select & Load your 4.0.2 dataset Folder:</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_load2
])

tab.children = [tab_load1, tab_load2]
tab.set_title(0, "Load Sec 4.0.1 Data")
tab.set_title(1, "Load Sec 4.0.2 Data")
display(tab)

### 4.1 Calculate TFIDF (by post_id or user) <font color="#FF0000">(Execute & Configure)</font>

In [ ]:
# 4.1 Calculate TFIDF
client.restart() #Restart Dask
RANGE = 1
idf_groupby_column = ""
target_groupby_column = ""
columnsToIgnore = ['content']
filtered_columns = ""

def loadColumns_1(btn):
  with open(CSV_PATH[0], 'r', encoding='utf-8-sig') as file:
    columns = [column.strip('\n') for column in file.readline().split(',')]
    columnsDropdown_1.options = columns
    print(f"Successfully loaded the columns")

def loadColumns_2(btn):
  with open(CSV_PATH[0], 'r', encoding='utf-8-sig') as file:
    columns = [column.strip('\n') for column in file.readline().split(',')]
    columnsDropdown_2.options = columns
    print(f"Successfully loaded the columns")

def loadColumns_3(btn):
  with open(CSV_PATH[0], 'r', encoding='utf-8-sig') as file:
    columns = [column.strip('\n') for column in file.readline().split(',')]
    columnsDropdown_3.options = columns
    print(f"Successfully loaded the columns")

def selectColumn_idf(btn):
  global idf_groupby_column
  idf_groupby_column = columnsDropdown_1.value
  print(f"Calculate IDF according to {idf_groupby_column}")

def selectColumn_target(btn):
  global target_groupby_column
  target_groupby_column = columnsDropdown_2.value
  print(f"Set Target Column to {target_groupby_column}")

def addColumnsToIgnore(btn):
  global columnsToIgnore
  columnsToIgnore.append(columnsDropdown_3.value)
  print(f"Columns to be Ignored: {columnsToIgnore}")

def resetColumns(btn):
  global columnsToIgnore
  columnsToIgnore = ['content']
  print(f"Columns to be Ignored: {columnsToIgnore}")

def postTFIDF(btn):
  clear_output(wait=True)
  for CSV in tqdm(list(CSV_PATH)[RANGE-1:], desc = f"Conversion Progress", ncols=100):
  # for CSV in track(list(CSV_PATH)[RANGE-1:], description=f"Conversion Progress"):
    # Initislization
    filtered_columns = columnsToIgnore.copy()
    for x in ['content', idf_groupby_column, target_groupby_column]:
      if x in filtered_columns:
        filtered_columns.remove(x)
    dest_filename = path.basename(CSV).split('.')[0] + '.csv'
    destination = path.join(DEST_PATH, dest_filename)
    df = pd.read_csv(CSV, encoding='utf-8-sig')
    df = df.drop(filtered_columns, axis=1)
    df['content'] = df['content'].swifter.apply(segmented_df_tfidf)


    try:
      df_idf = df.swifter.groupby([idf_groupby_column], as_index=False)['content']\
        .apply(lambda x: " ".join(x))['content'] # Unable to use swifter  
    except:
      df_idf = df.groupby([idf_groupby_column], as_index=False)['content']\
        .apply(lambda x: " ".join(x))['content'] # Unable to use swifter  
    #Groupby column for tfidf/cfidf
    content_index = list(df.columns).index('content')
    cols = [col for col in df.columns if col != 'content']

    try:
      df_target = df.swifter.groupby([target_groupby_column], as_index=False)[cols]\
        .apply(lambda x: x.head(1))\
          .reset_index(drop=True)\
            .sort_values(by=[target_groupby_column])\
              .reset_index(drop=True)
    except:
      df_target = df.groupby([target_groupby_column], as_index=False)[cols]\
        .apply(lambda x: x.head(1))\
          .reset_index(drop=True)\
            .sort_values(by=[target_groupby_column])\
              .reset_index(drop=True)
    # content = df.swifter.groupby(['post_id'], as_index=False).agg({'content': " ".join})['content']
    try:
      content = df.swifter.groupby([target_groupby_column], as_index=False)['content']\
        .apply(lambda x: " ".join(x))\
          .reset_index(drop=True)\
            .sort_values(by=[target_groupby_column])['content']\
              .reset_index(drop=True)
    except:
      content = df.groupby([target_groupby_column], as_index=False)['content']\
        .apply(lambda x: " ".join(x))\
          .reset_index(drop=True)\
            .sort_values(by=[target_groupby_column])['content']\
              .reset_index(drop=True)
    try:         
      df_target.insert(content_index, 'content', content)
    except:
      print("Insert column exception raised due to differnt logic of groupby, no need insert")

    channels = df_idf

    posts = df_target['content']

    for x in ['content', idf_groupby_column, target_groupby_column]:
      if x in columnsToIgnore:
        df_target = df_target.drop(x, axis=1)

    #Word Count Vector
    cv = CountVectorizer()
    word_count_vector = cv.fit_transform(channels) #Generate word counts for idf

    #Calculate idf using Word Count Vector
    tfidf_transformer = TfidfTransformer(use_idf=True)
    tfidf_transformer.fit(word_count_vector) # Calculate channel idf
    count_vector = cv.transform(posts) #Generate word count matrix of docs
    tfidf_score = tfidf_transformer.transform(count_vector)
    temp = pd.DataFrame(tfidf_score.T.todense(), index=cv.get_feature_names_out(), columns=df_target[target_groupby_column]).T
    tfidfList = []

    for index in range(temp.shape[0]):
      tfidfList.append(temp.iloc[index].sort_values(ascending=False)[:10].to_dict())
    df_target.insert(len(df_target.columns), 'tfidf', tfidfList)
    df_target.to_csv(destination, encoding='utf-8-sig', index=False)

# =======================================================
# =======================================================
# =======================================================

# Configure & Reload Widgets
# columnsDropdown = Dropdown([], 'Select Column: ')

btn_loadColumns_1 = button('Load Columns!')
btn_loadColumns_2 = button('Load Columns!')
btn_loadColumns_3 = button('Load Columns!')
btn_selectColumn_idf = button('Select Column!')
btn_selectColumn_target = button('Select Column!')
btn_addColumn_output = button('Add to drop list!')
btn_resetColumn_output = button('Reset!')
btn_setDest = button('Set Path')
btn_setRange = button('Set Range!')
btn_exeFx = button('Execute!')

btn_loadColumns_1.on_click(loadColumns_1)
btn_loadColumns_2.on_click(loadColumns_2)
btn_loadColumns_3.on_click(loadColumns_3)
btn_selectColumn_idf.on_click(selectColumn_idf)
btn_selectColumn_target.on_click(selectColumn_target)
btn_addColumn_output.on_click(addColumnsToIgnore)
btn_resetColumn_output.on_click(resetColumns)
btn_setDest.on_click(setDestination)
btn_setRange.on_click(setRange)
btn_exeFx.on_click(postTFIDF)

input_boundedInt = boundedInt()

tab_selectColumn_idf = VBox([
  widgets.HTML(value="<b><font style='color:red'>This section will group the data according to your selection</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>e.g selected 'channel', then then 'content' will be grouped by 'channel'</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>Select column to calculate IDF:</font></b>"),
  btn_loadColumns_1,
  columnsDropdown_1,
  btn_selectColumn_idf
])

tab_selectColumn_target = VBox([
  widgets.HTML(value="<b><font style='color:red'>This section is to define your data level</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>e.g you want to get post-level tfidf, then select 'post_id' as target</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>Select target column:</font></b>"),
  btn_loadColumns_2,
  columnsDropdown_2,
  btn_selectColumn_target
])

tab_setOutputFormat = VBox([
  widgets.HTML(value="<b><font style='color:red'>This section will set the output format of your work</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>By default 'content' column will not be included in output file</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>Drop unwanted columns: </font></b>"),
  btn_loadColumns_3,
  columnsDropdown_3,
  HBox([
  btn_addColumn_output,
  btn_resetColumn_output
  ])
])

tab_setDest = VBox([
  widgets.HTML(value="Select Output Destination: <b><font style='color:red'>Make sure you have created a folder first!</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_setDest
  ])

tab_setRange = VBox([
  widgets.HTML(value="Set the Starting Position for processing: <b>Default processing the <font style='color:red'>FIRST</font> file</b>"),
  input_boundedInt,
  btn_setRange
])

tab_execute = VBox([
  widgets.HTML(value='<b>If Everything is ready, Click the button below:</b>'),
  btn_exeFx
])

#Tab
tab.children = [tab_selectColumn_idf, tab_selectColumn_target, tab_setOutputFormat, tab_setDest, tab_setRange, tab_execute]
for index, title in enumerate(["Set IDF Column", "Set Target Column", "Set Output Format"] \
  + tabTitles[3:]):
  
  tab.set_title(index, title)

display(tab)

### 4.2 Calculate CFIDF (by post_id or user) <font color="#FF0000">(Execute & Configure)</font>

In [ ]:
# 4.2 Calculate CFIDF
client.restart() #Restart Dask
RANGE = 1
idf_groupby_column = ""
target_groupby_column = ""
columnsToIgnore = ['content']
filtered_columns = ""

def loadColumns_1(btn):
  with open(CSV_PATH2[0], 'r', encoding='utf-8-sig') as file:
    columns = [column.strip('\n') for column in file.readline().split(',')]
    columnsDropdown_1.options = columns
    print(f"Successfully loaded the columns")

def loadColumns_2(btn):
  with open(CSV_PATH2[0], 'r', encoding='utf-8-sig') as file:
    columns = [column.strip('\n') for column in file.readline().split(',')]
    columnsDropdown_2.options = columns
    print(f"Successfully loaded the columns")

def loadColumns_3(btn):
  with open(CSV_PATH2[0], 'r', encoding='utf-8-sig') as file:
    columns = [column.strip('\n') for column in file.readline().split(',')]
    columnsDropdown_3.options = columns
    print(f"Successfully loaded the columns")

def selectColumn_idf(btn):
  global idf_groupby_column
  idf_groupby_column = columnsDropdown_1.value
  print(f"Calculate IDF according to {idf_groupby_column}")

def selectColumn_target(btn):
  global target_groupby_column
  target_groupby_column = columnsDropdown_2.value
  print(f"Set Target Column to {target_groupby_column}")

def addColumnsToIgnore(btn):
  global columnsToIgnore
  columnsToIgnore.append(columnsDropdown_3.value)
  print(f"Columns to be Ignored: {columnsToIgnore}")

def resetColumns(btn):
  global columnsToIgnore
  columnsToIgnore = ['content']
  print(f"Columns to be Ignored: {columnsToIgnore}")

def cfidfColumns(row):
  columns = {}
  # columns['post_id'] = row.name
  # columns['cfidf_concept'] = row[conceptDict.keys()].sort_values(ascending=False).to_dict() # Old
  cfidf_concept_values = [row[x] if x in row else 0 for x in conceptDict.keys()]
  cfidf_concepts = {k:v for k,v in zip(conceptDict.keys(), cfidf_concept_values)}
  # columns['cfidf_concept'] = {k:v for k,v in sorted(cfidf_concepts.items(), key=lambda item: item[1], reverse=True)}
  # columns['cfidf'] = row.sort_values(ascending=False)[:10].to_dict()
  # for key, value in conceptDict.items():
  #   columns[key] = row[key]
  columns.update(cfidf_concepts)
  return pd.Series(columns)

def postCFIDF(btn):
  for CSV in tqdm(list(CSV_PATH2)[RANGE-1:], desc = f"Conversion Progress", ncols= 100):
  # Initialization
    filtered_columns = columnsToIgnore.copy()

    for x in ['content', idf_groupby_column, target_groupby_column]:
      if x in filtered_columns:
        filtered_columns.remove(x)

    dest_filename = path.basename(CSV).split('.')[0] + '.csv'
    destination = path.join(DEST_PATH, dest_filename)
    # df = mpd.read_csv(CSV, encoding='utf-8-sig')
    df = pd.read_csv(CSV, encoding='utf-8-sig')
    df = df.drop(filtered_columns, axis=1)
    # npContent = np.array(df['content'])
    # df['content'] = [segment_df_cfidf(x) for x in npContent]
    df['content'] = df['content'].swifter.apply(join_segmented_df_cfidf)
    #Groupby channel
    # df_idf = df.groupby(['channel'], as_index=False).agg({'channel': 'first', 'content': ' '.join})
    try:
      df_idf = df.swifter.groupby([idf_groupby_column], as_index=False)['content']\
        .apply(lambda x: " ".join(x))['content']
    except:
      df_idf = df.groupby([idf_groupby_column], as_index=False)['content']\
        .apply(lambda x: " ".join(x))['content']
    #Groupby column for tfidf/cfidf
    content_index = list(df.columns).index('content')
    cols = [col for col in df.columns if col != 'content']
    
    try:
      df_target = df.swifter.groupby([target_groupby_column], as_index=False)[cols]\
        .apply(lambda x: x.head(1))\
          .reset_index(drop=True)\
            .sort_values(by=[target_groupby_column])\
              .reset_index(drop=True)
    except:
      df_target = df.groupby([target_groupby_column], as_index=False)[cols]\
        .apply(lambda x: x.head(1))\
          .reset_index(drop=True)\
            .sort_values(by=[target_groupby_column])\
              .reset_index(drop=True)
      
    try:
      content = df.swifter.groupby([target_groupby_column], as_index=False)['content']\
        .apply(lambda x: " ".join(x))\
          .reset_index(drop=True)\
            .sort_values(by=[target_groupby_column])['content']\
              .reset_index(drop=True)
    except:
      content = df.groupby([target_groupby_column], as_index=False)['content']\
        .apply(lambda x: " ".join(x))\
          .reset_index(drop=True)\
            .sort_values(by=[target_groupby_column])['content']\
              .reset_index(drop=True)
    
    try:
      df_target.insert(content_index, 'content', content)
    except:
      print("Insert column exception raised due to differnt logic of groupby, no need insert")
    #tfidf (post-level by date)
    # Here, maybe not necessary to copy df_idf & df_target['content'] again for the operation below
    # channels = df_idf
    posts = df_target['content']

    for x in ['content', idf_groupby_column, target_groupby_column]:
      if x in columnsToIgnore:
        df_target = df_target.drop(x, axis=1)
    #Word Count Vector
    cv = CountVectorizer()
    # df_idf == channels
    word_count_vector = cv.fit_transform(df_idf) #Generate word counts for idf 

    #Calculate idf using Word Count Vector
    tfidf_transformer = TfidfTransformer(use_idf=True)
    tfidf_transformer.fit(word_count_vector) # Calculate channel idf
    # df_target['content'] == posts
    count_vector = cv.transform(posts) #Generate word count matrix of docs
    tfidf_score = tfidf_transformer.transform(count_vector)
    temp = pd.DataFrame(tfidf_score.T.todense(), index=cv.get_feature_names_out()\
      , columns=df_target[target_groupby_column]).T.reset_index(drop=True)
    
    df_output = pd.concat([df_target, temp.swifter.apply(cfidfColumns, axis=1)], axis=1)
    # temp2.insert(1, "channel", df.channel)
    df_output.to_csv(destination, encoding='utf-8-sig', index=False)

# Configure & Reload Widgets
# columnsDropdown = Dropdown([], 'Select Column: ')

btn_loadColumns_1 = button('Load Columns!')
btn_loadColumns_2 = button('Load Columns!')
btn_loadColumns_3 = button('Load Columns!')
btn_selectColumn_idf = button('Select Column!')
btn_selectColumn_target = button('Select Column!')
btn_addColumn_output = button('Add to drop list!')
btn_resetColumn_output = button('Reset!')
btn_setDest = button('Set Path')
btn_setRange = button('Set Range!')
btn_exeFx = button('Execute!')

btn_loadColumns_1.on_click(loadColumns_1)
btn_loadColumns_2.on_click(loadColumns_2)
btn_loadColumns_3.on_click(loadColumns_3)
btn_selectColumn_idf.on_click(selectColumn_idf)
btn_selectColumn_target.on_click(selectColumn_target)
btn_addColumn_output.on_click(addColumnsToIgnore)
btn_resetColumn_output.on_click(resetColumns)
btn_setDest.on_click(setDestination)
btn_setRange.on_click(setRange)
btn_exeFx.on_click(postCFIDF)

input_boundedInt = boundedInt()

tab_selectColumn_idf = VBox([
  widgets.HTML(value="<b><font style='color:red'>This section will group the data according to your selection</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>e.g selected 'channel', then then 'content' will be grouped by 'channel'</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>Select column to calculate IDF:</font></b>"),
  btn_loadColumns_1,
  columnsDropdown_1,
  btn_selectColumn_idf
])

tab_selectColumn_target = VBox([
  widgets.HTML(value="<b><font style='color:red'>This section is to define your data level</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>e.g you want to get post-level cfidf, then select 'post_id' as target</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>Select target column:</font></b>"),
  btn_loadColumns_2,
  columnsDropdown_2,
  btn_selectColumn_target
])

tab_setOutputFormat = VBox([
  widgets.HTML(value="<b><font style='color:red'>This section will set the output format of your work</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>By default 'content' column will not be included in output file</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>Drop unwanted columns: </font></b>"),
  btn_loadColumns_3,
  columnsDropdown_3,
  HBox([
  btn_addColumn_output,
  btn_resetColumn_output
  ])
])

tab_setDest = VBox([
  widgets.HTML(value="Select Output Destination: <b><font style='color:red'>Make sure you have created a folder first!</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_setDest
  ])

tab_setRange = VBox([
  widgets.HTML(value="Set the Starting Position for processing: <b>Default processing the <font style='color:red'>FIRST</font> file</b>"),
  input_boundedInt,
  btn_setRange
])

tab_execute = VBox([
  widgets.HTML(value='<b>If Everything is ready, Click the button below:</b>'),
  btn_exeFx
])

tab.children = [tab_selectColumn_idf, tab_selectColumn_target, tab_setOutputFormat, tab_setDest, tab_setRange, tab_execute]
for index, title in enumerate(["Set IDF Column", "Set Target Column", "Set Output Format"] \
  + tabTitles[3:]):
  
  tab.set_title(index, title)

display(tab)

### 4.3 Calculate Number of Concept Terms from extracted Terms (by post_id or user) <font color="#FF0000">(Execute & Configure)</font>

In [ ]:
# 4.3 Calculate number of concepts terms
client.restart() #Restart Dask
RANGE = 1
target_groupby_column = ""
columnsToIgnore = ['content']
filtered_columns = ""

def loadColumns_2(btn):
  with open(CSV_PATH2[0], 'r', encoding='utf-8-sig') as file:
    columns = [column.strip('\n') for column in file.readline().split(',')]
    columnsDropdown_2.options = columns
    print(f"Successfully loaded the columns")

def loadColumns_3(btn):
  with open(CSV_PATH2[0], 'r', encoding='utf-8-sig') as file:
    columns = [column.strip('\n') for column in file.readline().split(',')]
    columnsDropdown_3.options = columns
    print(f"Successfully loaded the columns")

def selectColumn_target(btn):
  global target_groupby_column
  target_groupby_column = columnsDropdown_2.value
  print(f"Set Target Column to {target_groupby_column}")

def addColumnsToIgnore(btn):
  global columnsToIgnore
  columnsToIgnore.append(columnsDropdown_3.value)
  print(f"Columns to be Ignored: {columnsToIgnore}")

def resetColumns(btn):
  global columnsToIgnore
  columnsToIgnore = ['content']
  print(f"Columns to be Ignored: {columnsToIgnore}")

def countTermsCFIDF(btn):
  for CSV in tqdm(list(CSV_PATH2)[RANGE-1:], desc='Progress: ', ncols=100):

    filtered_columns = columnsToIgnore.copy()
    for x in ['content', target_groupby_column]:
      if x in filtered_columns:
        filtered_columns.remove(x)

    dest_filename = path.basename(CSV).split('.')[0] + '.csv'
    destination = path.join(DEST_PATH, dest_filename)
    # Counting Terms
    df = pd.read_csv(CSV, encoding='utf-8-sig')
    df = df.drop(filtered_columns, axis=1)

    df['content'] = df['content'].swifter.apply(segmented_replaceTerms_cfidf)

    content_index = list(df.columns).index('content')
    cols = [col for col in df.columns if col != 'content']

    try:
      df_target = df.swifter.groupby([target_groupby_column], as_index=False)[cols]\
        .apply(lambda x: x.head(1))\
          .reset_index(drop=True)\
            .sort_values(by=[target_groupby_column])\
              .reset_index(drop=True)
    except:
      df_target = df.groupby([target_groupby_column], as_index=False)[cols]\
        .apply(lambda x: x.head(1))\
          .reset_index(drop=True)\
            .sort_values(by=[target_groupby_column])\
              .reset_index(drop=True)
 
    content = df.groupby([target_groupby_column], as_index=False)\
      .agg({'content': sum})['content']

    try:
      df_target.insert(content_index, 'content', content)
    except:
      print("Insert column exception raised due to differnt logic of groupby, no need insert")
    # df = df.groupby('post_id', as_index=False).agg({'post_id': 'first', 'channel': 'first', 'content': sum})
    df_termsCounting = df_target['content']\
      .swifter.apply(countTerms)\
        .swifter.apply(cfidfTermsCounting)
        
    for x in ['content', target_groupby_column]:
      if x in columnsToIgnore:
        df_target = df_target.drop(x, axis=1)

    # Concat

    df_output = pd.concat([df_target, df_termsCounting], axis=1)
    df_output.to_csv(destination, encoding='utf-8-sig', index=False)


# Configure & Reload Widgets
# columnsDropdown = Dropdown([], 'Select Column: ')

btn_loadColumns_2 = button('Load Columns!')
btn_loadColumns_3 = button('Load Columns!')
btn_selectColumn_target = button('Select Column!')
btn_addColumn_output = button('Add Column!')
btn_resetColumn_output = button('Reset!')
btn_setRange = button('Set Range')
btn_setDest = button('Set Path')
btn_exeFx = button('Execute!')

btn_loadColumns_2.on_click(loadColumns_2)
btn_loadColumns_3.on_click(loadColumns_3)
btn_selectColumn_target.on_click(selectColumn_target)
btn_addColumn_output.on_click(addColumnsToIgnore)
btn_resetColumn_output.on_click(resetColumns)
btn_setDest.on_click(setDestination)
btn_setRange.on_click(setRange)
btn_exeFx.on_click(countTermsCFIDF)

input_boundedInt = boundedInt()


tab_selectColumn_target = VBox([
  widgets.HTML(value="<b><font style='color:red'>This section is to define your data level</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>e.g you want to calculate post-level number of concept terms, then select 'post_id' as target</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>Select target column:</font></b>"),
  btn_loadColumns_2,
  columnsDropdown_2,
  btn_selectColumn_target
])

tab_setOutputFormat = VBox([
  widgets.HTML(value="<b><font style='color:red'>This section will set the output format of your work</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>By default 'content' column will not be included in output file</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>Drop unwanted columns: </font></b>"),
  btn_loadColumns_3,
  columnsDropdown_3,
  HBox([
  btn_addColumn_output,
  btn_resetColumn_output
  ])
])

tab_setDest = VBox([
  widgets.HTML(value="Select Output Destination: <b><font style='color:red'>Make sure you have created a folder first!</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_setDest
  ])

tab_execute = VBox([
  widgets.HTML(value='<b>If Everything is ready, Click the button below:</b>'),
  btn_exeFx
])

tab_setRange = VBox([
  widgets.HTML(value="Set the Starting Position for processing: <b>Default processing the <font style='color:red'>FIRST</font> file</b>"),
  input_boundedInt,
  btn_setRange
])

#Tab
tab.children = [tab_selectColumn_target, tab_setOutputFormat, tab_setDest, tab_setRange, tab_execute]
for index, title in enumerate(["Set Target Column", "Set Output Format"] \
  + tabTitles[3:]):
  
  tab.set_title(index, title)

display(tab)

### 4.4 Calculate Number of Comments contain Concepts (by post_id or user) <font color="#FF0000">(Execute & Configure)</font>

In [ ]:
# 4.4 Calculate number of comments contain concepts
client.restart() #Restart Dask
RANGE = 1
target_groupby_column = ""
columnsToIgnore = ['content']
filtered_columns = ""

def loadColumns_2(btn):
  with open(CSV_PATH2[0], 'r', encoding='utf-8-sig') as file:
    columns = [column.strip('\n') for column in file.readline().split(',')]
    columnsDropdown_2.options = columns
    print(f"Successfully loaded the columns")

def loadColumns_3(btn):
  with open(CSV_PATH2[0], 'r', encoding='utf-8-sig') as file:
    columns = [column.strip('\n') for column in file.readline().split(',')]
    columnsDropdown_3.options = columns
    print(f"Successfully loaded the columns")

def selectColumn_target(btn):
  global target_groupby_column
  target_groupby_column = columnsDropdown_2.value
  print(f"Set Target Column to {target_groupby_column}")

def addColumnsToIgnore(btn):
  global columnsToIgnore
  columnsToIgnore.append(columnsDropdown_3.value)
  print(f"Columns to be Ignored: {columnsToIgnore}")

def resetColumns(btn):
  global columnsToIgnore
  columnsToIgnore = ['content']
  print(f"Columns to be Ignored: {columnsToIgnore}")

def countCommentsCFIDF(btn):
  # global df_columns, df_commentsCounting
  for CSV in tqdm(list(CSV_PATH2)[RANGE-1:], desc='Coversion Progress', ncols=100):
    
    filtered_columns = columnsToIgnore.copy()
    for x in ['content', target_groupby_column]:
      if x in filtered_columns:
        filtered_columns.remove(x)

    dest_filename = path.basename(CSV).split('.')[0] + '.csv'
    destination = path.join(DEST_PATH, dest_filename)
    # Counting Terms
    df = pd.read_csv(CSV, encoding='utf-8-sig')
    df = df.drop(filtered_columns, axis=1)
    # npContent = np.array(df['content'])
    # df['content'] = [segmentation_cfidf(x) for x in npContent] # Segment terms & Replace Concepts
    df['content'] = df['content'].swifter.apply(segmented_replaceTerms_cfidf)
    cfidf_comments_columns = df['content'].swifter.apply(cfidfCommentsCounting)
    df = pd.concat([df, cfidf_comments_columns], axis=1)

    cols = [col for col in df.columns if col not in cfidf_comments_columns.columns]
    try:
      df_target = df.swifter.groupby([target_groupby_column], as_index=False)[cols]\
        .apply(lambda x: x.head(1))\
            .sort_values(by=[target_groupby_column])\
              .reset_index(drop=True)[cols]
    except:
      df_target = df.groupby([target_groupby_column], as_index=False)[cols]\
        .apply(lambda x: x.head(1))\
            .sort_values(by=[target_groupby_column])\
              .reset_index(drop=True)[cols]

    try:
      df_columns = df.swifter.groupby([target_groupby_column], as_index=False)[cfidf_comments_columns.columns]\
        .apply(lambda x: x.sum())\
          .sort_values(by=[target_groupby_column])\
            .reset_index(drop=True)[cfidf_comments_columns.columns]
    except:
      df_columns = df.groupby([target_groupby_column], as_index=False)[cfidf_comments_columns.columns]\
        .apply(lambda x: x.sum())\
          .sort_values(by=[target_groupby_column])\
            .reset_index(drop=True)[cfidf_comments_columns.columns]
    
    cols = [col for col in df.columns if col != 'content']
    for x in ['content', target_groupby_column]:
      if x in columnsToIgnore:
        df_target = df_target.drop(x, axis=1)

    df_output = pd.concat([df_target, df_columns], axis=1)
    df_output.to_csv(destination, encoding='utf-8-sig', index=False)
    # df = pd.concat([df['post_id'], df_columns], axis=1)
    # df_commentsCounting = df.groupby(['post_id'], as_index=False).sum()
    # df_commentsCounting = df_commentsCounting.drop(['post_id'], axis=1)
    # Read tfidf content
    # content_index = list(df.columns).index('content')
    # cols = [col for col in df.columns if col != 'content']

    # df_cfidf = pd.read_csv(CFIDF_CSV, encoding='utf-8-sig')
    # # Concat
    # df_concat = pd.concat([df_cfidf, df_commentsCounting], axis=1)
    # df_concat.to_csv(destination, encoding='utf-8-sig', index=False)

foldersOnly = list(filter(lambda x: path.isdir(x), listdir(DIR)))
#Widgets
# columnsDropdown = Dropdown([], 'Select Column: ')

btn_loadColumns_2 = button('Load Columns!')
btn_loadColumns_3 = button('Load Columns!')
btn_selectColumn_target = button('Select Column!')
btn_addColumn_output = button('Add Column!')
btn_resetColumn_output = button('Reset!')
btn_setDest = button('Set Path')
btn_setRange = button('Set Range')
btn_exeFx = button('Execute!')

btn_loadColumns_2.on_click(loadColumns_2)
btn_loadColumns_3.on_click(loadColumns_3)
btn_selectColumn_target.on_click(selectColumn_target)
btn_addColumn_output.on_click(addColumnsToIgnore)
btn_resetColumn_output.on_click(resetColumns)
btn_setRange.on_click(setRange)
btn_setDest.on_click(setDestination)
btn_exeFx.on_click(countCommentsCFIDF)

input_boundedInt = boundedInt()

tab_selectColumn_target = VBox([
  widgets.HTML(value="<b><font style='color:red'>This section is to define your data level</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>e.g you want to calculate post-level number of concept comments, then select 'post_id' as target</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>Select target column:</font></b>"),
  btn_loadColumns_2,
  columnsDropdown_2,
  btn_selectColumn_target
])

tab_setOutputFormat = VBox([
  widgets.HTML(value="<b><font style='color:red'>This section will set the output format of your work</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>By default 'content' column will not be included in output file</font></b>"),
  widgets.HTML(value="<b><font style='color:red'>Drop unwanted columns: </font></b>"),
  btn_loadColumns_3,
  columnsDropdown_3,
  HBox([
  btn_addColumn_output,
  btn_resetColumn_output
  ])
])

tab_setDest = VBox([
  widgets.HTML(value="Select Output Destination: <b><font style='color:red'>Make sure you have created a folder first!</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_setDest
  ])

tab_setRange = VBox([
  widgets.HTML(value="Set the Starting Position for processing: <b>Default processing the <font style='color:red'>FIRST</font> file</b>"),
  input_boundedInt,
  btn_setRange
])

tab_execute = VBox([
  widgets.HTML(value='<b>If Everything is ready, Click the button below:</b>'),
  btn_exeFx
])

#Tab
tab.children = [tab_selectColumn_target, tab_setOutputFormat, tab_setDest, tab_setRange, tab_execute]
for index, title in enumerate(["Set Target Column", "Set Output Format"] \
  + tabTitles[3:]):
  
  tab.set_title(index, title)

display(tab)

## 5. Post-processing <font color="#FF0000">(Execute & Configure)</font>

### 5.1 Merge splitted CSVs into single CSV! (optional) (e.g merge 183 days) <font color="#FF0000">(Execute & Configure)</font>

In [ ]:
# 5.1 merge splitted CSVs
fileName = None
# Set File Name & Path
def setFileName(btn):
  global fileName
  fileName = path.join(folder_pathDisplay1.value, input_Text.value + ".csv")
  print(f"Successfully set file name to {input_Text.value}")

def mergeToSingleCSV(btn):
  print('Processing')
  with open(fileName, 'w', encoding='utf-8-sig') as file:
    for p_index, CSV in tqdm(list(enumerate(CSV_PATH)), desc="Progress: ", ncols=100):
      with open(CSV, encoding="utf-8-sig") as f:
          for idx, line in enumerate(f):
              if p_index == 0 and idx == 0:
                file.write(line)
              elif idx>0:
                file.write(line)
  print('Done')

# Configure & Reload Widgets
input_Text = inputText('File name:')

btn_load1 = button('Load Source!')
btn_setFileName = button('Set!')
btn_exeFx = button('Execute!')

btn_load1.on_click(loadDataset)
btn_setFileName.on_click(setFileName)
btn_exeFx.on_click(mergeToSingleCSV)

tab_load1 = VBox([
  widgets.HTML(value="<b><font style='color:red'>Select & Load your dataset Folder:</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_load1
])

tab_setFileName = VBox([
  widgets.HTML(value="<b>Default output to asset folder!!</b>"),
  widgets.HTML(value="<b><font style='color:red'>Set output filename:</font></b>"),
  input_Text,
  btn_setFileName
])

tab_execute = VBox([
  widgets.HTML(value='<b>If Everything is ready, Click the button below:</b>'),
  btn_exeFx
])

tab.children = [tab_load1, tab_setFileName, tab_execute]
for index, title in enumerate([tabTitles2[0],tabTitles2[1], tabTitles2[-1]]):
  tab.set_title(index, title)
display(tab)

### 5.2 Combine different Outputs into single CSV! (completed section 5.1 first!) (e.g combine post-level & user-level) <font color="#FF0000">(Execute & Configure)</font>

In [ ]:
# 5.2 combine different outputs
RANGE = 1
CSV_LIST = []
columnsToDrop = []
dropHeader = False
fileName = None

def addFilesToMerge(btn):
  global CSV_LIST
  filePath = path.join(folder_pathDisplay1.value, file_dropdown1.value)
  CSV_LIST.append(filePath)
  print(f"Data to be Merged: {CSV_LIST}")

def resetFiles(btn):
  global CSV_LIST
  CSV_LIST.clear()
  print(f"Data to be Merged: {CSV_LIST}")

def setFileName(btn):
  global fileName
  fileName = path.join(folder_pathDisplay1.value, input_Text.value + ".csv")
  print(f"Successfully set file name to {input_Text.value}")
  
def loadColumns(btn):
  columns = []
  for csv in CSV_LIST:
    df = pd.read_csv(csv, encoding="utf-8-sig")
    columns.extend(df.columns.values.tolist())
  columnsDropdown.options = [*set(columns)]

def addDropColumn(btn):
  global columnsToDrop
  columnsToDrop.append(columnsDropdown.value)
  print(F"Columns to be dropped: {columnsToDrop}")

def resetDropColumn(btn):
  global columnsToDrop
  columnsToDrop.clear()
  print("Columns to be dropped has beend cleared!!")

def combineCSVs(btn):
  print('Processing')
  dfs = [pd.read_csv(csv, encoding='utf-8-sig') for csv in CSV_LIST]
  df = pd.concat(dfs, axis=1)
  df = df.loc[:,~df.T.duplicated(keep='first')]
  print(headersDropdown.value)
  if len(columnsToDrop) != 0: 
    df = df.drop(columnsToDrop, axis=1)
  if headersDropdown.value == "Yes":
    df.to_csv(fileName, encoding='utf-8-sig', index=False, header=False)
  else:
    df.to_csv(fileName, encoding='utf-8-sig', index=False, header=True)
    # df = df.drop(index=df.index[0], axis=0, inplace=True)
  print('Done')

foldersOnly = list(filter(lambda x: path.isdir(x), listdir(DIR)))
#Widgets
input_Text = inputText('File name:')

columnsDropdown = Dropdown([], 'Column: ')
headersDropdown = Dropdown(["Yes", "No"], 'Option: ')
headersDropdown.value = "No"

btn_addFile_output = button('Add File!')
btn_resetFile_output = button('Reset!')
btn_loadColumns = button('Load Output Columns')
btn_addDropColumns = button('Add Columns')
btn_resetDropColumns = button('Reset Columns')
btn_setFileName = button('Set!')
# btn_setRange = button('Set Range')
btn_exeFx = button('Execute!')

btn_addFile_output.on_click(addFilesToMerge)
btn_resetFile_output.on_click(resetFiles)
btn_loadColumns.on_click(loadColumns)
btn_addDropColumns.on_click(addDropColumn)
btn_resetDropColumns.on_click(resetDropColumn)
btn_setFileName.on_click(setFileName)
# btn_setRange.on_click(setRange)
btn_exeFx.on_click(combineCSVs)

input_boundedInt = boundedInt()

tab_addFileToMerge = VBox([
  widgets.HTML(value="Add file that you want to merge in sequence:"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  HBox([
  btn_addFile_output,
  btn_resetFile_output,
  ])
])

tab_setFileName = VBox([
  widgets.HTML(value="<b>Default output to asset folder!!</b>"),
  widgets.HTML(value="<b><font style='color:red'>Set output filename:</font></b>"),
  input_Text,
  btn_setFileName
])

tab_dropColumn = VBox([
  widgets.HTML(value="<b><font style='color:red'>Only load the columns when you need to drop the output columns!!</font></b>"),
  btn_loadColumns,
  widgets.HTML(value="<b><font style='color:red'>Select the column that your want to drop!!</font></b>"),
  columnsDropdown,
  HBox([
  btn_addDropColumns,
  btn_resetDropColumns
  ])
])

tab_execute = VBox([
  widgets.HTML(value="<b><font style='color:red'>Do you want to keep the header (Column Names)? Keep header by default.</font>:</b>"),
  headersDropdown,
  widgets.HTML(value="<b>If Everything is ready, Click the button below:</b>"),
  btn_exeFx
])

tab.children = [tab_addFileToMerge, tab_setFileName, tab_dropColumn, tab_execute]
for index, title in enumerate(["Add Files", tabTitles2[1], "Drop Columns"] + tabTitles2[7:]):
  tab.set_title(index, title)

display(tab)